In [1]:
import spacy
from spacy import displacy
import numpy as np
import pandas as pd

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
from google.colab import files


uploaded = files.upload()


Saving heuristics.csv to heuristics.csv


In [4]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['heuristics.csv']),header=None)
print(df)


                                                    0
0   Where immediately before the issue of a notifi...
1   Where in respect of an industrial dispute rela...
2   At the time of hearing, the authority may dire...
3   The authority referred to in sub-section (6) m...
4   Finality of orders constituting Board, Committ...
5   Chief Inspector - The State Government may, by...
6   Where the applicant succeeds in such proceedin...
7   Penalty for obstructing Inspector. -Whoever wi...
8   Protection to persons acting under the Act. - ...
9   Transfer of cases from one Court to another. -...
10  Removal of difficulties.-If any difficulty ari...
11  Provided that the appropriate Government may, ...
12  Provided that in relation to the State of Jamm...
13  Provided further that when the provisions of t...
14  Every employee shall be entitled to be paid by...
15  Subject to the other provisions of this Act, e...
16  Where in respect of any accounting year referr...
17  In computing the allocab

In [5]:
df.head()

,0
0,Where immediately before the issue of a notifi...
1,Where in respect of an industrial dispute rela...
2,"At the time of hearing, the authority may dire..."
3,The authority referred to in sub-section (6) m...
4,"Finality of orders constituting Board, Committ..."


In [6]:
for index,row in df.iterrows():
  sentence = row.values[0]
  doc = nlp(sentence)
  for token in doc:
    ancestors = [t.text for t in token.ancestors]
    children = [t.text for t in token.children]
    if(token.dep_ == "ROOT"):
      print(token.text, "\t", token.i, "\t", 
          token.pos_, "\t", token.dep_, "\t", 
          ancestors, "\t", children)
  print("\n")

be 	 75 	 AUX 	 ROOT 	 [] 	 ['were', ',', 'under', ',', 'then', ',', 'notwithstanding', ',', 'wages', 'shall', 'payable', 'deemed', '.']


apply 	 133 	 VERB 	 ROOT 	 [] 	 ['is', 'is', 'then', ',', 'notwithstanding', ',', 'rates', 'shall', 'not', 'to', 'during', 'and', 'made']


direct 	 9 	 VERB 	 ROOT 	 [] 	 ['At', ',', 'authority', 'may', 'employers', 'deposit', '.']
paid 	 36 	 VERB 	 ROOT 	 [] 	 ['amount', 'may', 'be', 'to']


allow 	 41 	 VERB 	 ROOT 	 [] 	 ['referred', 'may', ',', 'is', ',', 'preferred']


Finality 	 0 	 NOUN 	 ROOT 	 [] 	 ['of', '-', 'committee-']
called 	 38 	 VERB 	 ROOT 	 [] 	 ['order', 'shall', 'be', 'in', 'in', 'called']


appoint 	 10 	 VERB 	 ROOT 	 [] 	 ['Inspector', 'Government', 'may', ',', 'by', 'person', '.']


calculate 	 14 	 VERB 	 ROOT 	 [] 	 ['succeeds', ',', 'authority', 'shall', 'amount', 'but', 'for', ',', 'and', 'direct', '.']
be 	 75 	 AUX 	 ROOT 	 [] 	 ['shall', ',', 'without', ',', 'recoverable', '.']


Penalty 	 0 	 NOUN 	 ROOT 	 [] 	 [

Protection 	 0 	 NOUN 	 ROOT 	 [] 	 ['to', '.', '-']
lie 	 19 	 VERB 	 ROOT 	 [] 	 ['suit', 'shall', 'against', 'for', '.']


In [ ]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    children = [t.text for t in token.children]
    if(token.dep_ == "ROOT"):
      print(token.text, "\t", token.i, "\t", 
          token.pos_, "\t", token.dep_, "\t", 
          ancestors, "\t", children)

Protection 	 0 	 NOUN 	 ROOT 	 [] 	 ['to', '.', '-']
lie 	 19 	 VERB 	 ROOT 	 [] 	 ['suit', 'shall', 'against', 'for', '.']


In [ ]:
sentence = "Protection to persons acting under the Act. - No suit, prosecution or other legal proceeding whatsoever shall lie against any person for anything which is in good faith done or intended to be done in due discharge of his duties under this Act."

In [ ]:
doc = nlp(sentence)

In [ ]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    children = [t.text for t in token.children]
    if(token.dep_ == "ROOT"):
      print(token.text, "\t", token.i, "\t", 
          token.pos_, "\t", token.dep_, "\t", 
          ancestors, "\t", children)

Protection 	 0 	 NOUN 	 ROOT 	 [] 	 ['to', '.', '-']
lie 	 19 	 VERB 	 ROOT 	 [] 	 ['suit', 'shall', 'against', 'for', '.']


In [ ]:
def find_root_of_sentence(doc):
    root_token = None
    for token in doc:
        if (token.dep_ == "ROOT"):
            root_token = token
    return root_token

In [ ]:
root_token = find_root_of_sentence(doc)

In [ ]:
def find_other_verbs(doc, root_token):
    other_verbs = []
    for token in doc:
        ancestors = list(token.ancestors)
        if (token.pos_ == "VERB" and len(ancestors) == 1\
            and ancestors[0] == root_token):
            other_verbs.append(token)
    return other_verbs

In [ ]:
other_verbs = find_other_verbs(doc, root_token)

In [ ]:
def get_clause_token_span_for_verb(verb, doc, all_verbs):
    first_token_index = len(doc)
    last_token_index = 0
    this_verb_children = list(verb.children)
    for child in this_verb_children:
        if (child not in all_verbs):
            if (child.i < first_token_index):
                first_token_index = child.i
            if (child.i > last_token_index):
                last_token_index = child.i
    return(first_token_index, last_token_index)

In [ ]:
token_spans = []   
all_verbs = [root_token] + other_verbs
for other_verb in all_verbs:
    (first_token_index, last_token_index) = \
     get_clause_token_span_for_verb(other_verb, 
                                    doc, all_verbs)
    token_spans.append((first_token_index, 
                        last_token_index))

In [ ]:
sentence_clauses = []
for token_span in token_spans:
    start = token_span[0]
    end = token_span[1]
    if (start < end):
        clause = doc[start:end]
        sentence_clauses.append(clause)
sentence_clauses = sorted(sentence_clauses, 
                          key=lambda tup: tup[0])

In [ ]:
clauses_text = [clause.text for clause in sentence_clauses]
print(clauses_text)

['suit, prosecution or other legal proceeding whatsoever shall lie against any person for anything which is in good faith done or intended to be done in due discharge of his duties under this Act']


VAISHNAVI'S AIM-ATTRIBUTE-DEONTIC

In [ ]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('merge_noun_chunks')
# text = "Every rule made under this Act shall be laid, as soon as maybe after it is made, before each House of Parliament, while it is in session, for a total period of thirty days which may be comprised in one Session or in two or more successive sessions, and if, before the expiry of the session immediately following the session or the successive sessions aforesaid. both Houses agree in making any modification in the rule or both Houses agree that the rule should not be made, the rule shall thereafter have effect only in such modified form or be of no effect, as the case may be; so, however, that any such modification or annulment shall be without prejudice to the validity of anything previously done under that rule."
# doc = nlp(text)
i = 0
noun_found = False
attribute = ""
deontic = ""
aim = ""
for text in clauses_text:
  doc = nlp(text)
  for entity in doc:
      print(entity, entity.pos_, entity.dep_)
      if (entity.pos_ == "PROPN" or entity.pos_ == "NOUN"):
          attribute = str(entity)
          aim = str(entity.head.text)
          aim_found = True
          break

  for entity in doc:
    if (entity.pos_ == "AUX") and (aim == entity.head.text):
      deontic = str(entity)
      # print(deontic + entity.head.text)

# print(doc.vocab)

# for chunk in doc.noun_chunks:
#     if str(chunk.text) == attribute:
#         aim = str(chunk.root.head.text)

print("attribute: " + attribute)
print("aim: " + aim)
print("deontic: " + deontic)

suit NOUN nsubj
attribute: suit
aim: lie
deontic: shall


# **REPLICATING ONLINE SPACY**

DHRUV'S AIM-ATTRIBUTE-DEONTIC

In [ ]:
nlp = spacy.load("en_core_web_sm")
sentence = ""
doc = nlp(sentence)


def merge_phrases(doc):
    with doc.retokenize() as retokenizer:
        for np in list(doc.noun_chunks):
            attrs = {
                "tag": np.root.tag_,
                "lemma": np.root.lemma_,
                "ent_type": np.root.ent_type_,
            }
            retokenizer.merge(np, attrs=attrs)
    return doc

def merge_punct(doc):
    spans = []
    for word in doc[:-1]:
        if word.is_punct or not word.nbor(1).is_punct:
            continue
        start = word.i
        end = word.i + 1
        while end < len(doc) and doc[end].is_punct:
            end += 1
        span = doc[start:end]
        spans.append((span, word.tag_, word.lemma_, word.ent_type_))
    with doc.retokenize() as retokenizer:
        for span, tag, lemma, ent_type in spans:
            attrs = {"tag": tag, "lemma": lemma, "ent_type": ent_type}
            retokenizer.merge(span, attrs=attrs)
    return doc

# Merge noun phrases into one token.
doc = merge_phrases(doc)
# Attach punctuation to tokens
doc = merge_punct(doc)

In [ ]:
for token in doc:
  ancestors = [t.text for t in token.ancestors]
  ancestors_token = [t for t in token.ancestors]
  children = [t.text for t in token.children]
  children_token = [t for t in token.children]
  left = [t.text for t in token.lefts]
  right = [t.text for t in token.rights]
  print(token.text, token.pos_, token.dep_,ancestors,children,left,right)


In [ ]:
ners = [(e,type(e)) for e in doc.ents]
ners

[]

In [ ]:
nlp.pipeline
# Tagger identifies verb, noun etc
# Parser identifies relationships b/w different tokens
# ner labels named entities

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7f9a5f665280>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x7f9a5f6656a0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7f9a5fcbdd60>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f9a5f72c100>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7f9a5f7d7700>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f9a5f969eb0>)]

In [ ]:
attribute = []
attribute_token = []
aim = []
aim_token = []
deontic = []
deontic_token = []
aim_2 = []
aim_2_token = []


For Attribute

In [ ]:
i = 0
for entity in doc:
  print(entity)
  if ((entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or "PRON") and (entity.dep_ == "nsubjpass" or entity.dep_ == "nsubj")):
    attribute.append(entity.text)
    attribute_token.append(entity)
    for attr in entity.children:
      if attr.dep_ == 'conj' and (attr.pos_ == "PROPN" or attr.pos_ == "NOUN"):
        attribute.append(attr.text)
        attribute_token.append(attr)
    
  if(len(attribute)>0):
    break

In [ ]:
print(attribute)

[]


For Deontic

In [ ]:
for entity in doc:
  print(entity)
  if ((entity.pos_ == "AUX") and (entity.dep_ == "aux" or entity.dep_ == "auxpass")):
    deontic.append(entity.text)
    deontic_token.append(entity)
    break
  
  

In [ ]:
print(deontic)

[]


For Aim

In [ ]:
for entity in doc:
  if (entity.pos_ =='NOUN' or entity.pos_ =='VERB'):
    for child in entity.children:
      if (child in attribute_token) and (child.dep_ == 'nsubj' or child.dep_ =='nsubjpass'):
        aim.append(entity.text)
        aim_token.append(entity)

        for attr in entity.children:
          if attr.dep_ == 'conj' and (attr.pos_ == "NOUN" or attr.pos_ == "VERB"):
            aim.append(attr.text)
            aim_token.append(attr)
  
  if(len(aim)>0):
    break

In [ ]:
print(aim)

[]


Aim Updated code

In [ ]:
for entity in doc:
  if (entity.pos_ =='VERB'):
    for child in entity.children:
      if (child in deontic_token) and (child.dep_ == 'aux'):
        aim_2.append(entity.text)
        aim_2_token.append(entity)

        for attr in entity.children:
          if attr.dep_ == 'conj' and (attr.pos_ == "NOUN" or attr.pos_ == "VERB"):
            aim.append(attr.text)
            aim_token.append(attr)
  
  if(len(aim_2)>0):
    break

In [ ]:
print(aim_2)

[]


##Testing on Dataset

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

# import gspread
# from google.auth import default
# creds, _ = default()

# gc = gspread.authorize(creds)
# sheet = gc.open_by_url('SHEET LINK HERE')


In [ ]:
# worksheet = sheet.worksheet('SUBSHEET NAME HERE')
# rows = worksheet.get_all_values()
# import pandas as pd
# df = pd.DataFrame.from_records(rows)

In [ ]:
# df
# df = df.drop(index=0)
# df.columns = ["Sentence","Attribute","Aim","Deontic","Context","B","O","Attribute_test","Aim_test","Deontic_test"]
# attr1 = []
# aim1 = []
# deo1 = []

# for sentence in df["Sentence"]:
#   doc = nlp(sentence)
#   # Merge noun phrases into one token.
#   doc = merge_phrases(doc)
#   # Attach punctuation to tokens
#   doc = merge_punct(doc)
#   for token in doc:
#     ancestors = [t.text for t in token.ancestors]
#     ancestors_token = [t for t in token.ancestors]
#     children = [t.text for t in token.children]
#     children_token = [t for t in token.children]
#     left = [t.text for t in token.lefts]
#     right = [t.text for t in token.rights]
#   ners = [(e,type(e)) for e in doc.ents]
#   nlp.pipeline
#   attribute = []
#   attribute_token = []
#   aim = []
#   aim_token = []
#   deontic = []
#   deontic_token = []
#   #attributes
#   i = 0
#   for entity in doc:
#     if ((entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or "PRON") and (entity.dep_ == "nsubjpass" or entity.dep_ == "nsubj")):
#       attribute.append(entity.text)
#       attribute_token.append(entity)
#       for attr in entity.children:
#         if attr.dep_ == 'conj' and (attr.pos_ == "PROPN" or attr.pos_ == "NOUN"):
#           attribute.append(attr.text)
#           attribute_token.append(attr)
      
#     if(len(attribute)>0):
#       break
  
#   for entity in doc:
#     print(entity)
#     if ((entity.pos_ == "AUX") and (entity.dep_ == "aux" or entity.dep_ == "auxpass")):
#       deontic.append(entity.text)
#       deontic_token.append(entity)
#       break
  
#   for entity in doc:
#     if (entity.pos_ =='NOUN' or entity.pos_ =='VERB'):
#       for child in entity.children:
#         if (child in attribute_token) and (child.dep_ == 'nsubj' or child.dep_ =='nsubjpass'):
#           aim.append(entity.text)
#           aim_token.append(entity)

#           for attr in entity.children:
#             if attr.dep_ == 'conj' and (attr.pos_ == "NOUN" or attr.pos_ == "VERB"):
#               aim.append(attr.text)
#               aim_token.append(attr)
    
#     if(len(aim)>0):
#       break
#   attr1.append(attribute)
#   aim1.append(aim)
#   deo1.append(deontic)

In [ ]:
# df["Attribute_test"] = attr1
# df["Aim_test"] = aim1
# df["Deontic_test"] = deo1

In [ ]:
# sh = gc.create('A new spreadsheet')

# # Open our new sheet and add some data.
# # worksheet = gc.open('A new spreadsheet').sheet1
# sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1iS9MeDIB4JVgpig5s4x3PArDaO81CGEpjokSQNDJhyE/edit#gid=0')
# worksheet = sheet.worksheet('Sheet2')


# df.to_csv('test_results.csv', index = False)

